In [ ]:
import torch
import numpy as np
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import rc

%matplotlib inline

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)

In [ ]:
alpha = 0.2
beta = 0.8
xs = np.linspace(0, 1, 1000)

def iou_weight(xs, *, sigma=0.3):
    return np.exp(-(xs ** 2) / sigma)

def emb_weight(xs, *, center, sigma):
    return 1 - np.exp(-((xs - center) ** 2) / sigma)
    return 1 - (1 / (1 + np.exp(-(sim * temperature) * (xs - center))))

t = 0.01
center = (alpha + beta) / 2
emb_sigma = -((alpha - center) ** 2) / np.log(t)
iou_ys = iou_weight(xs)
emb_ys = emb_weight(xs, center=0.5, sigma=emb_sigma)
weight_ys = iou_ys * emb_ys

fig, ax = plt.subplots(figsize=(16, 6), nrows=1, ncols=1)

ax.set_title('Embedding Similarity + Soft-NMS Weighting Function')
ax.set_xlabel('IoU Score')
ax.set_ylabel('Weight')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.vlines(alpha, 0, 1, color='black', alpha=0.5, linestyles='dashed')
ax.vlines(beta, 0, 1, color='black', alpha=0.5, linestyles='dashed')
ax.plot(xs, iou_ys, label='IoU weight')
ax.plot(xs, emb_ys, label='Similarity weight')
ax.plot(xs, weight_ys, linewidth=3, label='Final (combined) weight')
ax.legend(loc='best')

fig.show()